In [ ]:
#In this file PDF content (TEXT, ABSTRACT) will be extracted
# As example only one of most famous article
"Attention to detail" #is used to produce summary

In [1]:
import unstructured
from unstructured.partition.pdf import partition_pdf

In [ ]:
base_dir = "Article_contents"

for i in range(1,2):
    article_folder = f"{base_dir}/Article_{i}"
    pdf_path = f"{article_folder}/Article {i}.pdf"
    images_folder = f"{article_folder}/Images"
    document_order_path = f"{article_folder}/Document_Order.txt"
    print("working on",pdf_path)

    elements = partition_pdf(
        filename=pdf_path,                 
        strategy="hi_res",                  
        extract_images_in_pdf=True,         
        extract_image_block_types=["Image", "Table"], 
        extract_image_block_to_payload=False,  
        extract_image_block_output_dir=images_folder
    )
    element_extraction = process_elements(elements)

    with open(document_order_path, "w", encoding="utf-8") as f:
        f.write(element_extraction["Document_order"])
        print("file has been extracted")

    print("Images and text extracted for:",pdf_path)

working on Article_contents/Article_1/Article 1.pdf


In [8]:
def process_elements(elements):
    # Initialize lists to categorize elements
    Header = []
    Title = []
    Footer = []
    Text = []
    List_item = []
    Narrative_text = []
    Document_order = []
    abstract = []

    for i, element in enumerate(elements):
        # Get the next elements, if they exist
        next_ele = elements[i + 1] if i + 1 < len(elements) else None
        next_ele1 = elements[i + 2] if i + 2 < len(elements) else None

        if "unstructured.documents.elements.Header" in str(type(element)):
            Header.append(str(element))

        elif "unstructured.documents.elements.Footer" in str(type(element)):
            Footer.append(str(element))

        elif "unstructured.documents.elements.Title" in str(type(element)):
            title = str(element)

            # Checking for "Abstract" and extracting one or two NarrativeText paragraphs
            if title == "Abstract":
                if next_ele and "unstructured.documents.elements.NarrativeText" in str(type(next_ele)):
                    abstract.append(str(next_ele))
                if next_ele1 and "unstructured.documents.elements.NarrativeText" in str(type(next_ele1)):
                    abstract.append(str(next_ele1))

            if title == "Conclusion":
                Title.append(title)
                Document_order.append(title)
                if next_ele:
                    Document_order.append(str(next_ele))
                print("Conclusion found, exiting loop.")
                break

            # Process numeric titles
            if title and title[0].isdigit():
                Title.append(title)
                Document_order.append(title)

        elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
            Narrative_text.append(str(element))
            Document_order.append(str(element))
        elif "unstructured.documents.elements.Text" in str(type(element)):
            Text.append(str(element))
            Document_order.append(str(element))
        elif "unstructured.documents.elements.ListItem" in str(type(element)):
            List_item.append(str(element))
            Document_order.append(str(element))

    # Combine the document order into a single string
    Document_order = ' '.join(Document_order)
    
    return {
        "Header": Header,
        "Title": Title,
        "Footer": Footer,
        "Text": Text,
        "List_item": List_item,
        "Narrative_text": Narrative_text,
        "Document_order": Document_order,
        "Abstract": abstract
    }

In [ ]:
#Here we are using sematic Chunking from langchain
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document

text_splitter = SemanticChunker(OpenAIEmbeddings(),number_of_chunks=10, min_chunk_size=1500)
text_docs = text_splitter.create_documents([Document_order])
print(len(text_docs))